# Question 3
Name: AmirHossein Kargaran Khouzani
<br>
Student Number: 99201119

## part 1,2

In [335]:
import numpy as np
import time

In [179]:
def create_fake_data(N, d):
    # returns X array with shape (N, d)
    return 100 * np.random.rand(N, d)
    

def cosine_distance(x, y):
    # returns cosine distance between x and y
    if x.shape != y.shape:
        raise RuntimeError("input {} length not match {}".format(x.shape, y.shape))
    x_norm = np.linalg.norm(x)
    y_norm = np.linalg.norm(y)
    
    similiarity = np.dot(x, y.T)/(x_norm * y_norm) 
    distance = 1 - similiarity
    return distance
    

def find_k_nearest_neighbours(X, q, k):
    # returns k-nearest-neighbours of vector q (least distance with vector q)
    X_splitted = np.array_split(X, X.shape[0])
    return sorted(X_splitted, key = lambda v: cosine_distance(v, q))[:k]

### part 2.2: speed of computation for k = 20, N=10000, d=200

In [192]:
N = 10000
d = 200
k = 20
X = create_fake_data(N, d)
q = create_fake_data(1, d)

start_time = time.time()
find_k_nearest_neighbours(X, q, k)
(time.time() - start_time)

0.5349962711334229

## part 3 

In [178]:
class Signer:
    def __init__(self, f, d):
        # initiates f random vectors with dimension = d
        self.hyper_planes = []
        for i in range(0,f):
            h = np.random.uniform(low = -1.0, size = d)
            self.hyper_planes.append(h / np.linalg.norm(h))
        
    def hash_point(self, x):
        # returns list of numbers in {0, 1} -> right and left can be determined by inner product
        return [1 if np.dot(x, h) >= 0 else 0 for h in self.hyper_planes]

## part 4

In [360]:
class LSHIndex:
    def __init__(self, f, b, d):
        # initiates empty hash tables  
        self.b = b
        self.r = f // self.b
        self.f = self.r * self.b
        self.signer_object = Signer(self.f, d)
        self.hash_table = {k: {} for k in range(0, self.f, self.r)}
        self.X = {}
   
    def index(self, key, x):
        # returns a boolean True, indicating success
        self.X[key] = x
        hash_point = self.signer_object.hash_point(x)
        
        for b_index in range(0, self.f, self.r):
            next_b_index = b_index + self.r
            b_key = ''.join(map(str, hash_point[b_index : next_b_index]))
            
            try:
                self.hash_table[b_index][b_key].add(key)
            except:
                self.hash_table[b_index][b_key] = set()
                self.hash_table[b_index][b_key].add(key)
          
        return True
    
    def query(self, q, k):
        # returns k-approximate-nearest-neighbours of vector q
        hash_point_q = self.signer_object.hash_point(q)
        key_candidates = set()

        for b_index in range(0, self.f, self.r):
            next_b_index = b_index + self.r
            b_key = ''.join(map(str, hash_point_q[b_index : next_b_index]))

            if(b_key in self.hash_table[b_index]):
                key_candidates = key_candidates.union(self.hash_table[b_index][b_key])
                
        return find_k_nearest_neighbours(np.array([self.X[key] for key in key_candidates]), q, k)

## variables

In [365]:
N = 100000 #100000
d = 300 #300
k = 250 #250

b = 20 #2
r = 5 #5
f = b * r #100

X = create_fake_data(N, d)
q = create_fake_data(1, d)

iteration = 1

## lsh avergae speed

In [366]:
lsh_model = LSHIndex(f, b, d)
for index, value in enumerate(X):
    lsh_model.index(index, value)

In [367]:
start_time = time.time()

for i in range(0, iteration):
    result1 = lsh_model.query(q, k)
    
(time.time() - start_time)/iteration

7.1459949016571045

## bruteforce average speed

In [368]:
iteration = 1
start_time = time.time()

for i in range(0, iteration):
    result2 = find_k_nearest_neighbours(X, q, k)
    
(time.time() - start_time)/iteration

8.060996294021606

## error

In [346]:
total_error = 0.0
for i,j in zip(result1, result2):
    total_error += abs(cosine_distance(i, q)-cosine_distance(j, q))
    
total_error/k

array([[0.]])